In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as skmts
import sklearn.preprocessing as skpre
import scipy.stats as scistat
import glob
plt.style.use('fivethirtyeight')
%matplotlib inline

# ONEIL vs. ALMANAC

In [2]:
df = pd.read_pickle('../data/processed_data/drugcomb_processed_data.pkl')

oneil_df = df[df['study_name']=='ONEIL']
almanac_df = df[df['study_name']=='ALMANAC']
triplet_list = sorted(set(oneil_df['triplet'])&set(almanac_df['triplet']))
ipair_list = sorted(set(oneil_df['ipair'])&set(almanac_df['ipair']))
cid_list = sorted(set(oneil_df['cid'])&set(almanac_df['cid']))
print(f'find overlapping triplet between ONEIL and ALMANAC dataset={len(triplet_list)}')
print(f'find overlapping drug pair between ONEIL and ALMANAC dataset={len(ipair_list)}')
print(f'find overlapping cell line between ONEIL and ALMANAC dataset={len(cid_list)}')

find overlapping triplet between ONEIL and ALMANAC dataset=83
find overlapping drug pair between ONEIL and ALMANAC dataset=46
find overlapping cell line between ONEIL and ALMANAC dataset=9


# External Validation: 

## case1: predict known triplet

In [13]:
df1 = almanac_df[almanac_df['triplet'].isin(triplet_list)]
print(f'df1={df1.shape}')

df1=(133, 28)


## case2: predict unknown triplet

In [3]:
df2 = almanac_df[~almanac_df['triplet'].isin(triplet_list)]
print(f'df2={df2.shape}')

df2=(72875, 28)


## case3: predict unknown cell (drup pair is known)

In [8]:
df3 = almanac_df[almanac_df['ipair'].isin(ipair_list)]
df3 = df3[~df3['cid'].isin(cid_list)]
print(f'df3={df3.shape}')

df3=(769, 28)


## case4: predict unknown drug pair (cell is known)

In [9]:
df4 = almanac_df[almanac_df['cid'].isin(cid_list)]
df4 = df4[~df4['ipair'].isin(ipair_list)]
print(f'df4={df4.shape}')

df4=(14324, 28)


# case5: predict all (including overlapping triplets)

In [10]:
df5 = almanac_df.copy()
print(f'df5={df5.shape}')

df5=(73008, 28)


# Save to file

In [11]:
def classify_synergy(df, col_str='synergy_loewe', debug=False):
    """
    return df with categorical labels

    # classify loewe score
    # synergistic: loewe > 10, --> 1
    # antigonistic: loewe < -10, --> 0
    # additive: -10 < loewe < 10 --> 2
    """
    # replace value
    rule = lambda x: 1 if x > 10 else 0 if x < -10 else 2
    df['synergy_label'] = df[col_str].apply(rule)

    # stats
    n_data = len(df)
    n_class0 = len(df[df[col_str]==0])
    n_class1 = len(df[df[col_str]==1])
    n_class2 = len(df[df[col_str]==2])

    # excluding additive samples
    df = df[df['synergy_label'] != 2]


    if debug:
        a = len(df)
        b = n_class0 + n_class1 + n_class2
        if a != b:
            raise ValueError(f'Error, total size={a}, but got {b}')
    return df

def transform_synergy(df, col_str='synergy_loewe', debug=False):
    """ return power transformed synergy scores"""
    a = df[col_str].head(1).values[0]

    # data transformation
    pt = skpre.PowerTransformer(method='yeo-johnson', standardize=True)
    mm = skpre.MinMaxScaler(feature_range=(0,1))
    scaled_arr = pt.fit_transform(df[col_str].values.reshape(-1,1))
    scaled_arr = mm.fit_transform(scaled_arr.reshape(-1,1))

    # overwrite original values
    df['synergy_label'] = scaled_arr
    b = df[col_str].head(1).values[0]

    if debug:
        if a == b:
            raise ValueError(f'Error, a==b before and after transformation')
    return df

In [12]:
for scenario, df in {'scenario1': df1, 'scenario2': df2, 'scenario3': df3, 'scenario4': df4, 'scenario5': df5}.items():
    df = classify_synergy(df, col_str='synergy_loewe')

    # select columns
    df.loc[:, 'tcga_id'] = pd.factorize(df.copy().loc[:, 'tcga abbrev'])[0]
    df.loc[:, 'tissue_id'] = pd.factorize(df.copy().loc[:, 'tissue_name'])[0]
    df.loc[:, 'region_label'] = pd.factorize(df.copy().loc[:, 'dose_region'])[0]
    h_list = ['triplet', 'icombo', 'ipair', 'cid', 'synergy_label', 'region_label', 'tcga_id', 'tissue_id',
              'dose_region', 'tcga abbrev', 'tissue_name', 'synergy_loewe']
    df = df[h_list]
    
    # save to file
    fname = '../data/external/ALMANAC_'+ scenario +'_classification.data.pkl'
    df.to_pickle(fname)

    print(f'save to file: {fname}')
    print(df['synergy_label'].value_counts())

/tmp/ipykernel_4055464/3332745448.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['synergy_label'] = df[col_str].apply(rule)
/tmp/ipykernel_4055464/3332745448.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['synergy_label'] = df[col_str].apply(rule)


save to file: ../data/external/ALMANAC_scenario1_classification.data.pkl
0    68
1    65
Name: synergy_label, dtype: int64
save to file: ../data/external/ALMANAC_scenario2_classification.data.pkl
0    37227
1    35648
Name: synergy_label, dtype: int64
save to file: ../data/external/ALMANAC_scenario3_classification.data.pkl
0    440
1    329
Name: synergy_label, dtype: int64
save to file: ../data/external/ALMANAC_scenario4_classification.data.pkl
0    7452
1    6872
Name: synergy_label, dtype: int64
save to file: ../data/external/ALMANAC_scenario5_classification.data.pkl
0    37295
1    35713
Name: synergy_label, dtype: int64


In [15]:
for scenario, df in {'scenario1': df1, 'scenario2': df2, 'scenario3': df3, 'scenario4': df4, 'scenario5': df5}.items():
    df = transform_synergy(df, col_str='synergy_loewe')

    # select columns
    df.loc[:, 'tcga_id'] = pd.factorize(df.copy().loc[:, 'tcga abbrev'])[0]
    df.loc[:, 'tissue_id'] = pd.factorize(df.copy().loc[:, 'tissue_name'])[0]
    df.loc[:, 'region_label'] = pd.factorize(df.copy().loc[:, 'dose_region'])[0]
    h_list = ['triplet', 'icombo', 'ipair', 'cid', 'synergy_label', 'region_label', 'tcga_id', 'tissue_id',
              'dose_region', 'tcga abbrev', 'tissue_name', 'synergy_loewe']
    df = df[h_list]
    
    # save to file
    fname = '../data/external/ALMANAC_'+ scenario +'_regression.data.pkl'
    df.to_pickle(fname)

    print(f'save to file: {fname}')
    print(df['synergy_label'].describe())

/tmp/ipykernel_4055464/3332745448.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['synergy_label'] = scaled_arr
/tmp/ipykernel_4055464/1578061612.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'tcga_id'] = pd.factorize(df.copy().loc[:, 'tcga abbrev'])[0]
/tmp/ipykernel_4055464/1578061612.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

save to file: ../data/external/ALMANAC_scenario1_regression.data.pkl
count    133.000000
mean       0.481590
std        0.231896
min        0.000000
25%        0.352646
50%        0.418868
75%        0.634914
max        1.000000
Name: synergy_label, dtype: float64
save to file: ../data/external/ALMANAC_scenario2_regression.data.pkl
count    72875.000000
mean         0.447860
std          0.151968
min          0.000000
25%          0.348677
50%          0.410121
75%          0.549787
max          1.000000
Name: synergy_label, dtype: float64
save to file: ../data/external/ALMANAC_scenario3_regression.data.pkl
count    769.000000
mean       0.466720
std        0.180863
min        0.000000
25%        0.355716
50%        0.435163
75%        0.594067
max        1.000000
Name: synergy_label, dtype: float64
save to file: ../data/external/ALMANAC_scenario4_regression.data.pkl
count    14324.000000
mean         0.454274
std          0.164387
min          0.000000
25%          0.348854
50%       

# Performance of the non-skilled model

In [2]:
case2_df = pd.read_pickle('../data/external/ALMANAC_scenario2_classification.data.pkl')
case2_df.head()

,triplet,icombo,ipair,cid,synergy_label,region_label,tcga_id,tissue_id,dose_region,tcga abbrev,tissue_name,synergy_loewe
1,"001, RAD_Sorafenib_LOX IMVI",did_5=did_1386=cid_82=r0.1=c2.0,did_5-did_1386,cid_82,0,0,0,0,high_low,TCGA_SKCM,skin,-44.778147
2,"001, RAD_Sorafenib_LOX IMVI",did_5=did_1386=cid_82=r0.1=c20.0,did_5-did_1386,cid_82,0,1,0,0,high,TCGA_SKCM,skin,-22.308577
0,"001, RAD_Sorafenib_SK-MEL-2",did_5=did_1386=cid_149=r0.1=c2.0,did_5-did_1386,cid_149,0,0,0,0,high_low,TCGA_SKCM,skin,-45.554533
0,"Crizotinib_001, RAD_SK-MEL-2",did_618=did_5=cid_149=r0.01=c0.006,did_618-did_5,cid_149,1,2,0,0,low,TCGA_SKCM,skin,19.384170
1,"Crizotinib_001, RAD_SK-MEL-2",did_618=did_5=cid_149=r0.01=c0.06,did_618-did_5,cid_149,1,3,0,0,low_high,TCGA_SKCM,skin,18.946444


In [3]:
case2_df['synergy_label'].value_counts()

0    37227
1    35648
Name: synergy_label, dtype: int64

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as skmts
import scipy.stats as scistat
import glob
plt.style.use('fivethirtyeight')

def eval_mts(y_list, y_pred_list, task_str):
    """return evaluation  metrices"""
    mask = ~np.isnan(y_pred_list)
    y_list = y_list[mask]
    y_pred_list = y_pred_list[mask]
    if task_str == 'classification':
        auc = skmts.roc_auc_score(y_list, y_pred_list, average='micro')
        aucprc = skmts.average_precision_score(y_list, y_pred_list, average='micro')
        y_pred_list = (y_pred_list>0.5).astype(int)
        acc = skmts.accuracy_score(y_list, y_pred_list)
        mcc = skmts.matthews_corrcoef(y_list, y_pred_list)
        f1 = skmts.f1_score(y_list, y_pred_list, average='micro')
        precision = skmts.precision_score(y_list, y_pred_list)
        recall = skmts.recall_score(y_list, y_pred_list)
        kappa = skmts.cohen_kappa_score(y_list, y_pred_list)
        balanced_acc = skmts.balanced_accuracy_score(y_list, y_pred_list)

        df = pd.DataFrame({'metrices': ['AUC', 'AUPRC', 'Accuracy', 'MCC', 'F1', 'Precision', 'Recall', 'Kappa', 'Balanced_Accuracy'],
                           'score': [auc, aucprc, acc, mcc, f1, precision, recall, kappa, balanced_acc]})

        tn, fp, fn, tp = skmts.confusion_matrix(y_list, y_pred_list).ravel()
        print(f'confusion matrix: TN={tn}, FP={fp}, FN={fn}, TP={tp}')

    elif task_str == 'regression':
        mae = skmts.mean_absolute_error(y_list, y_pred_list)
        mse = skmts.mean_squared_error(y_list, y_pred_list)
        rmse = skmts.mean_squared_error(y_list, y_pred_list, squared=False)
        r2 = skmts.r2_score(y_list, y_pred_list)
        pcc, pval = scistat.pearsonr(y_list, y_pred_list)
        spr, pval = scistat.spearmanr(y_list, y_pred_list)

        df = pd.DataFrame({'metrices': ['MAE', 'MSE', 'RMSE', 'R2', 'Pearson r', 'Spearman r'],
                           'score': [mae, mse, rmse, r2, pcc, spr]})
    else:
        raise ValueError(f'Error, {task_str} should be either classification or regression!!!')
    return df

In [7]:
eval_mts(case2_df['synergy_label'], np.array([0]*len(case2_df)), 'classification')

confusion matrix: TN=37227, FP=0, FN=35648, TP=0


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,metrices,score
0,AUC,0.500000
1,AUPRC,0.489166
2,Accuracy,0.510834
3,MCC,0.000000
4,F1,0.510834
5,Precision,0.000000
6,Recall,0.000000
7,Kappa,0.000000
8,Balanced_Accuracy,0.500000


In [9]:
case2_df['predicted_synergy'] = 0
case2_df

,triplet,icombo,ipair,cid,synergy_label,region_label,tcga_id,tissue_id,dose_region,tcga abbrev,tissue_name,synergy_loewe,predicted_synergy
1,"001, RAD_Sorafenib_LOX IMVI",did_5=did_1386=cid_82=r0.1=c2.0,did_5-did_1386,cid_82,0,0,0,0,high_low,TCGA_SKCM,skin,-44.778147,0
2,"001, RAD_Sorafenib_LOX IMVI",did_5=did_1386=cid_82=r0.1=c20.0,did_5-did_1386,cid_82,0,1,0,0,high,TCGA_SKCM,skin,-22.308577,0
0,"001, RAD_Sorafenib_SK-MEL-2",did_5=did_1386=cid_149=r0.1=c2.0,did_5-did_1386,cid_149,0,0,0,0,high_low,TCGA_SKCM,skin,-45.554533,0
0,"Crizotinib_001, RAD_SK-MEL-2",did_618=did_5=cid_149=r0.01=c0.006,did_618-did_5,cid_149,1,2,0,0,low,TCGA_SKCM,skin,19.384170,0
1,"Crizotinib_001, RAD_SK-MEL-2",did_618=did_5=cid_149=r0.01=c0.06,did_618-did_5,cid_149,1,3,0,0,low_high,TCGA_SKCM,skin,18.946444,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044330,thiotepa_temozolomide_K-562,did_1961=did_1945=cid_72=r1.0=c2.0,did_1961-did_1945,cid_72,0,2,10,1,low,TCGA_LCML,haematopoietic_and_lymphoid,-25.500809,0
2849930,thiotepa_temozolomide_LOX IMVI,did_1961=did_1945=cid_82=r1.0=c2.0,did_1961-did_1945,cid_82,0,2,0,0,low,TCGA_SKCM,skin,-38.261655,0
3678941,thiotepa_temozolomide_U251,did_1961=did_1945=cid_176=r10.0=c0.2,did_1961-did_1945,cid_176,0,0,7,7,high_low,TCGA_GBM,brain,-12.245231,0
4164894,thiotepa_thalidomide_HCT-15,did_1961=did_1955=cid_56=r10.0=c1.0,did_1961-did_1955,cid_56,0,1,5,5,high,TCGA_COAD,large_intestine,-11.722540,0


In [10]:
df = case2_df
# get triplet from icombo
df[['did_row', 'did_col', 'cid', 'conc_r', 'conc_c']]= df['icombo'].str.split('=', expand=True)
df['triplet'] = df['did_row'] + '_' + df['did_col'] + '_' + df['cid']
   
    
# collect qualified triplets: have positives and negatives  
g = df.groupby(by=['triplet'])['synergy_label'].agg('mean').to_frame('#label')
all_pos = g[g['#label']==1].index.tolist()
all_neg = g[g['#label']==0].index.tolist()
triplet_list = sorted(set(g.index) - set(all_pos+all_neg))
    
# get per-triplet performance
record_list = [] # (N, AUC, AUPRC)
grps = df.groupby(['triplet'])
for triplet in triplet_list:
    sample = grps.get_group(triplet)
    auc = skmts.roc_auc_score(sample['synergy_label'], sample['predicted_synergy'])
    auprc = skmts.average_precision_score(sample['synergy_label'], sample['predicted_synergy'])
    record_list.append( (len(sample), auc, auprc) )
df = pd.DataFrame.from_records(record_list, index=triplet_list, columns=['N', 'AUC', 'AUPRC'])
df = df.sort_values(by=['N', 'AUPRC', 'AUC'], ascending=False)
print(f'Top 10 triplet:\n{df.head(10)}')
    
# calculate mean , standard deviation
print(f'per-triplet performance')
print(pd.concat([df.mean(axis=0).to_frame(name='mean'), df.std(axis=0).to_frame(name='standard deviation')], axis=1))

Top 10 triplet:
                          N  AUC     AUPRC
did_653_did_836_cid_72   12  0.5  0.166667
did_1115_did_836_cid_72  11  0.5  0.545455
did_627_did_836_cid_150  10  0.5  0.800000
did_627_did_836_cid_147  10  0.5  0.600000
did_147_did_836_cid_74    9  0.5  0.111111
did_3_did_836_cid_119     7  0.5  0.857143
did_627_did_836_cid_124   7  0.5  0.857143
did_627_did_836_cid_176   7  0.5  0.857143
did_627_did_836_cid_93    7  0.5  0.857143
did_808_did_1211_cid_65   7  0.5  0.428571
per-triplet performance
           mean  standard deviation
N      2.912595            1.140266
AUC    0.500000            0.000000
AUPRC  0.500983            0.142179


In [4]:
df2.columns

Index(['drug_row', 'drug_col', 'cell_line_name', 'conc_r', 'conc_c',
       'inhibition', 'synergy_zip', 'synergy_loewe', 'synergy_hsa',
       'synergy_bliss', 'ic50_row', 'ic50_col', 'ri_row', 'ri_col',
       'tissue_name', 'study_name', 'drug_row_clinical_phase',
       'drug_col_clinical_phase', 'did_row', 'did_col', 'cid', 'triplet',
       'icombo', 'ipair', 'cate_r', 'cate_c', 'dose_region', 'tcga abbrev'],
      dtype='object')

In [6]:
len( set(df2['ipair'])&set(oneil_df['ipair']) )

46

In [7]:
len( set(df2['cid'])&set(oneil_df['cid']) )

9

In [8]:
len( set(df2['triplet'])&set(oneil_df['triplet']) )

0

In [5]:
len(df2['cid'].unique())

46

In [6]:
len( set(df2['tissue_name'])&set(oneil_df['tissue_name']) )

6

In [7]:
len(df2['tissue_name'].unique())

9

In [9]:
df2['tissue_name'].unique()

array(['skin', 'haematopoietic_and_lymphoid', 'kidney', 'lung', 'breast',
       'large_intestine', 'ovary', 'brain', 'prostate'], dtype=object)

In [10]:
set(df2['tissue_name'])&set(oneil_df['tissue_name'])

{'breast', 'large_intestine', 'lung', 'ovary', 'prostate', 'skin'}